In [1]:
import pandas as pd
import glob
from datetime import datetime
import os

In [2]:
os.getcwd()

'C:\\Users\\msaucedo'

In [3]:
os.chdir("S:/Projects/Intern Projects/Colorado Intern/3ce/GZIP FILES/2022")

In [4]:
df = pd.read_csv('all_data2022.csv', header = None)

In [5]:
#https://www.ncei.noaa.gov/pub/data/noaa/isd-lite/isd-lite-format.pdf
#Documentation for Column Names 
column_names = ["Year","Month", "Day", "Hour", "airTemp","dewPoint","pressure","windDirection","windSpeed","cloudCover","oneHourPrecipitation", "sixHourPercipitation", "StationID"]

In [6]:
#Setting column names defined above
df.columns = column_names

In [7]:
df.tail()

,Year,Month,Day,Hour,airTemp,dewPoint,pressure,windDirection,windSpeed,cloudCover,oneHourPrecipitation,sixHourPercipitation,StationID
19971,2022,6,17,4,140.0,80,-9999,250,72,-9999,-9999,-9999,749179-00392
19972,2022,6,17,5,130.0,70,-9999,250,36,-9999,-9999,-9999,749179-00392
19973,2022,6,17,6,130.0,80,-9999,220,51,-9999,-9999,-9999,749179-00392
19974,2022,6,17,7,130.0,70,-9999,230,57,-9999,-9999,-9999,749179-00392
19975,2022,6,17,8,120.0,80,-9999,230,31,-9999,-9999,-9999,749179-00392


In [8]:
#Change each datetime function to a string to combine column into one as Time Stamp 
#strp takes only string function

df[['Year', 'Month', 'Day', 'Hour']] = df[['Year', 'Month', 'Day', 'Hour']]. astype(str)

##Joining columns with
df['DateTime'] = df[['Month', 'Day','Year']].apply("-".join, axis=1)
df['DateTime'] = df[['DateTime', 'Hour']].apply(" ".join, axis=1)

In [9]:
df.head()

,Year,Month,Day,Hour,airTemp,dewPoint,pressure,windDirection,windSpeed,cloudCover,oneHourPrecipitation,sixHourPercipitation,StationID,DateTime
0,2022,1,1,1,106.0,61,10105,300,46,0,0,-9999,722897-93206,1-1-2022 1
1,2022,1,1,2,106.0,28,10111,330,36,0,0,-9999,722897-93206,1-1-2022 2
2,2022,1,1,3,100.0,33,10117,330,21,0,0,-9999,722897-93206,1-1-2022 3
3,2022,1,1,4,94.0,22,10126,320,46,0,0,-9999,722897-93206,1-1-2022 4
4,2022,1,1,5,100.0,33,10130,320,62,0,0,-9999,722897-93206,1-1-2022 5


In [10]:
#Helpful Time Series Documentation - https://www.geeksforgeeks.org/python-datetime-strptime-function/
df['DateTime'] = df['DateTime'].apply(lambda x:  datetime.strptime(x, "%m-%d-%Y %H"))

In [11]:
df.head()

,Year,Month,Day,Hour,airTemp,dewPoint,pressure,windDirection,windSpeed,cloudCover,oneHourPrecipitation,sixHourPercipitation,StationID,DateTime
0,2022,1,1,1,106.0,61,10105,300,46,0,0,-9999,722897-93206,2022-01-01 01:00:00
1,2022,1,1,2,106.0,28,10111,330,36,0,0,-9999,722897-93206,2022-01-01 02:00:00
2,2022,1,1,3,100.0,33,10117,330,21,0,0,-9999,722897-93206,2022-01-01 03:00:00
3,2022,1,1,4,94.0,22,10126,320,46,0,0,-9999,722897-93206,2022-01-01 04:00:00
4,2022,1,1,5,100.0,33,10130,320,62,0,0,-9999,722897-93206,2022-01-01 05:00:00


In [12]:
#Creating new a dataframe, reorgnized columns and got rid of redundant rows (hour, month, year, day) that's included in the time stamp 
df1 = df[["StationID", "DateTime","Year", "Month", "Day", "Hour", "airTemp","dewPoint","pressure","windDirection","windSpeed","cloudCover","oneHourPrecipitation", "sixHourPercipitation"]]

In [13]:
# function to see if a value is replaced with a missing operator
# Conversion of data

def checkForNone(reading):
    if reading in ("-9999", "-99999", "-999999", ""):
        return None
    else:
        return False

def convertMeasurementNoScaling(reading):
    if checkForNone(reading) is None:
        return None
    else:
        return float(reading)

# convert from deca to standard
def convertMeasurement(reading):
    if checkForNone(reading) is None:
        return None
    else:
        r = float(reading)
        return r / 10

# look up cloud cover label
def cloudCoverLookup(reading):
    return {
        "0": "None, SKC or CLR",
        "1": "One okta - 1/10 or less but not zero",
        "2": "Two oktas - 2/10 - 3/10, or FEW",
        "3": "Three oktas - 4/10",
        "4": "Four oktas - 5/10, or SCT",
        "5": "Five oktas - 6/10",
        "6": "Six oktas - 7/10 - 8/10",
        "7": "Seven oktas - 9/10 or more but not 10/10, or BKN",
        "8": "Eight oktas - 10/10, or OVC",
        "9": "Sky obscured, or cloud amount cannot be estimated",
        "10": "Partial obscuration",
        "11": "Thin scattered",
        "12": "Scattered",
        "13": "Dark scattered",
        "14": "Thin broken",
        "15": "Broken",
        "16": "Dark broken",
        "17": "Thin overcast",
        "18": "Overcast",
        "19": "Dark overcast",
    }.get(reading)

In [14]:
#applying function(s) to airtemp column 
df1["airTemp"] = df1["airTemp"].apply(convertMeasurement)
df1.head()

,StationID,DateTime,Year,Month,Day,Hour,airTemp,dewPoint,pressure,windDirection,windSpeed,cloudCover,oneHourPrecipitation,sixHourPercipitation
0,722897-93206,2022-01-01 01:00:00,2022,1,1,1,10.6,61,10105,300,46,0,0,-9999
1,722897-93206,2022-01-01 02:00:00,2022,1,1,2,10.6,28,10111,330,36,0,0,-9999
2,722897-93206,2022-01-01 03:00:00,2022,1,1,3,10.0,33,10117,330,21,0,0,-9999
3,722897-93206,2022-01-01 04:00:00,2022,1,1,4,9.4,22,10126,320,46,0,0,-9999
4,722897-93206,2022-01-01 05:00:00,2022,1,1,5,10.0,33,10130,320,62,0,0,-9999


In [15]:
df2 = df1[["StationID", "DateTime","Year", "Month", "Day", "Hour", "airTemp"]]

In [16]:
df2["TempF"] = df2["airTemp"] 

C:\Users\msaucedo\AppData\Local\Temp/ipykernel_12332/4020949513.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["TempF"] = df2["airTemp"]


In [17]:
df2.drop(["airTemp"], axis = 1) 

,StationID,DateTime,Year,Month,Day,Hour,TempF
0,722897-93206,2022-01-01 01:00:00,2022,1,1,1,10.6
1,722897-93206,2022-01-01 02:00:00,2022,1,1,2,10.6
2,722897-93206,2022-01-01 03:00:00,2022,1,1,3,10.0
3,722897-93206,2022-01-01 04:00:00,2022,1,1,4,9.4
4,722897-93206,2022-01-01 05:00:00,2022,1,1,5,10.0
...,...,...,...,...,...,...,...
19971,749179-00392,2022-06-17 04:00:00,2022,6,17,4,14.0
19972,749179-00392,2022-06-17 05:00:00,2022,6,17,5,13.0
19973,749179-00392,2022-06-17 06:00:00,2022,6,17,6,13.0
19974,749179-00392,2022-06-17 07:00:00,2022,6,17,7,13.0


In [18]:
df3 = df2[["StationID", "DateTime", "Year", "Month", "Day", "Hour", "TempF"]]
df3.dropna()

,StationID,DateTime,Year,Month,Day,Hour,TempF
0,722897-93206,2022-01-01 01:00:00,2022,1,1,1,10.6
1,722897-93206,2022-01-01 02:00:00,2022,1,1,2,10.6
2,722897-93206,2022-01-01 03:00:00,2022,1,1,3,10.0
3,722897-93206,2022-01-01 04:00:00,2022,1,1,4,9.4
4,722897-93206,2022-01-01 05:00:00,2022,1,1,5,10.0
...,...,...,...,...,...,...,...
19971,749179-00392,2022-06-17 04:00:00,2022,6,17,4,14.0
19972,749179-00392,2022-06-17 05:00:00,2022,6,17,5,13.0
19973,749179-00392,2022-06-17 06:00:00,2022,6,17,6,13.0
19974,749179-00392,2022-06-17 07:00:00,2022,6,17,7,13.0


In [19]:
def celtofar(celsius):
     
    temp_fahr = (celsius * 9/5) + 32 
    
    return temp_fahr

In [20]:
#applying function above
df3["TempF"] = celtofar(df3["TempF"])

C:\Users\msaucedo\AppData\Local\Temp/ipykernel_12332/3776773313.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3["TempF"] = celtofar(df3["TempF"])


In [21]:
df3.to_csv('3CEConvertedHistoricalData2022.csv', index = False)

In [22]:
df3.tail()

,StationID,DateTime,Year,Month,Day,Hour,TempF
19971,749179-00392,2022-06-17 04:00:00,2022,6,17,4,57.2
19972,749179-00392,2022-06-17 05:00:00,2022,6,17,5,55.4
19973,749179-00392,2022-06-17 06:00:00,2022,6,17,6,55.4
19974,749179-00392,2022-06-17 07:00:00,2022,6,17,7,55.4
19975,749179-00392,2022-06-17 08:00:00,2022,6,17,8,53.6
